In [70]:
from convnext import get_convnext
from interp_utils import register_hook, remove_hooks
from imagenet_val import ValData, simple_labels
import torch
import torch.nn as nn
import torch.nn.functional as F

val = ValData()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = get_convnext()
model.eval().to(device);

In [71]:
def label_smoothing_loss(logits, targets, smoothing=0.1):
    logprobs = F.log_softmax(logits, dim=-1)
    loss = (1-smoothing)*F.cross_entropy(logits, targets)-smoothing*logprobs.mean()
    return loss

In [89]:
def gelu_to_relu_layer(model, layer):
    remove_hooks(model, quiet=True)
    def gelu_to_relu(module, input, output):
        output = F.relu(input[0])
        return output
    handle = register_hook(model.blocks[layer].act, gelu_to_relu)
    return handle

In [96]:
from tqdm import tqdm

BATCH_SIZE = 100

ls_losses = torch.zeros(18)
ce_losses = torch.zeros(18)

for layer_idx in range(18):
    print(f"Layer {layer_idx}")
    handle = gelu_to_relu_layer(model, layer_idx)
    val_set_pbar = tqdm(val.as_batches(BATCH_SIZE), total=len(val)//BATCH_SIZE,)
    val_set_pbar.set_description(f"Layer {layer_idx}")
    ls_batch_losses = []
    ce_batch_losses = []
    for i, batch in enumerate(val_set_pbar):
        batch_data = batch.data.to(device)
        targets = torch.tensor(batch.class_ids).to(device)
        with torch.no_grad():
            logits = model(batch_data)
            ls_loss = label_smoothing_loss(logits, targets)
            ce_loss = F.cross_entropy(logits, targets)
        ls_batch_losses.append(ls_loss)
        ce_batch_losses.append(ce_loss)
        val_set_pbar.set_description(f"Layer {layer_idx} LS: {ls_loss:.3f} CE: {ce_loss:.3f}")
    ls_losses[layer_idx] = torch.stack(ls_batch_losses).mean()
    ce_losses[layer_idx] = torch.stack(ce_batch_losses).mean()


Layer 0


Layer 0 LS: 2.380 CE: 1.639: 100%|██████████| 500/500 [03:27<00:00,  2.41it/s]


Layer 1


Layer 1 LS: 2.508 CE: 1.781: 100%|██████████| 500/500 [03:37<00:00,  2.30it/s]


Layer 2


...

Layer 15 LS: 2.095 CE: 1.312: 100%|██████████| 500/500 [03:22<00:00,  2.47it/s]


Layer 16


Layer 16 LS: 2.095 CE: 1.311: 100%|██████████| 500/500 [03:21<00:00,  2.48it/s]


Layer 17


Layer 17 LS: 2.096 CE: 1.313: 100%|██████████| 500/500 [03:22<00:00,  2.47it/s]


In [98]:
remove_hooks(model)

BATCH_SIZE = 100


val_set_pbar = tqdm(val.as_batches(BATCH_SIZE), total=len(val)//BATCH_SIZE,)
val_set_pbar.set_description(f"Layer {layer_idx}")
ls_batch_losses = []
ce_batch_losses = []
for i, batch in enumerate(val_set_pbar):
    batch_data = batch.data.to(device)
    targets = torch.tensor(batch.class_ids).to(device)
    with torch.no_grad():
        logits = model(batch_data)
        ls_loss = label_smoothing_loss(logits, targets)
        ce_loss = F.cross_entropy(logits, targets)
    ls_batch_losses.append(ls_loss)
    ce_batch_losses.append(ce_loss)
    val_set_pbar.set_description(f"Without ablation | LS: {ls_loss:.3f} CE: {ce_loss:.3f}")

ls_loss = torch.stack(ls_batch_losses).mean()
ce_loss = torch.stack(ce_batch_losses).mean()


Without ablation. LS: 2.096 CE: 1.312: 100%|██████████| 500/500 [03:23<00:00,  2.45it/s]


In [114]:
data = {}
data['ls_losses_by_relud_layer'] = ls_losses.cpu().numpy()
data['ce_losses_by_relud_layer'] = ce_losses.cpu().numpy()
data['normal_ls_loss'] = ls_loss.cpu().numpy()
data['normal_ce_loss'] = ce_loss.cpu().numpy()

torch.save(data, 'gelu_to_relu_results.pt')